In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import bs4 as BeautifulSoup

from main import *

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the player statistics page
driver.get("https://www.nba.com/stats/players/traditional")

# Explicit wait to ensure the table is loaded
wait = WebDriverWait(driver, 10)

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

accept_cookies(driver)
time.sleep(3)

# Clicar en cierta columna (tipo de jugada) de un jugador en concreto
try:
    column_indices = {"FGA": 11, "3PA": 14, "REB": 21, "AST": 22, "TOV": 23, "STL": 24, "BLK": 25}

    link_xpath = general_players_table_xpath + f'tbody/tr[1]/td[{column_indices["FGA"]}]/a'
    link_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, link_xpath))
    )

    link_url = link_element.get_attribute("href")
    print(f"Link found: {link_element.get_attribute('href')}")
    driver.get(link_url)

except Exception as e:
    print(f"Error: {e}")


# Create the data frame for the labels of the plays
header_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/thead/tr/th'
header_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, header_xpath))
)

# Extraer los nombres de las columnas (headers)
headers = [header.text for header in header_elements]
headers.append("AST")  # Añadir la columna para el link del video
headers.append("Video Link")  # Añadir la columna para el link del video

# Crear el DataFrame vacío con los headers
labels_df = pd.DataFrame(columns=headers)

# Click on some row of the table of plays
video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

# Load all the rows in one page before iterating
dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
option_value = '-1'

dropdown_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
)
select = Select(dropdown_element)
select.select_by_value(option_value)


for i in range(1, 652):  # Iterate over all the plays of a particular player
    line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

    play = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, line_play))
    )

    columns_play = play.find_elements(By.XPATH, line_play)
    
    row_data = []
    for col in columns_play:
        row_data.append(col.text)
    
    play.click()

    # Verify if the shot is assisted
    ast_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/main/section[1]/div/div[2]/h2'

    ast_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ast_xpath))
    )
    ast_present = 'AST' in ast_element.text
    row_data.append(ast_present)

    # Video display element
    video_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, video_display_xpath))
    )
    video_src = video_element.get_attribute('src')
    row_data.append(video_src)  

    # La fila se agrega al DataFrame
    labels_df.loc[i - 1] = row_data  


driver.quit()

Cookies banner closed.
Link found: https://www.nba.com/stats/events/?CFID=&CFPARAMS=&ContextMeasure=FGA&GameID=&PlayerID=203507&Season=2024-25&SeasonType=Regular%20Season&TeamID=1610612749&flag=3&sct=plot&section=game
